In [11]:
from src.indexing_utils import load_docs, create_docstore
from ved_src import retreival_utils
import os

In [61]:
nodes = load_docs('../../../storage/docstore/swapped_summary_nodes', return_docstore=False)

In [6]:
for node in nodes:
    node.excluded_embed_metadata_keys.extend(['summary', 'is_summary_swapped'])

In [12]:
create_docstore(nodes=nodes, save_dir='../../../storage/docstore', store_name='swapped_summary_nodes')

In [42]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.llms.gemini import Gemini
from llama_index.core import Settings

Settings.embed_model = GeminiEmbedding(model_name='models/embedding-001')
Settings.llm = Gemini(model_name='models/gemini-pro', temperature=0.5)

In [16]:
# load the documents and create the index
index = VectorStoreIndex(nodes=nodes, embed_model=Settings.embed_model, show_progress=True)

# store it for later
index.storage_context.persist(persist_dir='../../../storage/vectorstore/summary_embeddings_vectorstore')

Generating embeddings:   0%|          | 0/427 [00:00<?, ?it/s]

In [29]:
store = load_docs('../../../storage/docstore/swapped_summary_nodes', return_docstore=True)

In [67]:
query_engine = index.as_query_engine(llm=Settings.llm, similarity_top_k=3)

In [64]:
type(query_engine)

llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine

In [68]:
response = query_engine.query("How does ayurveda deal with illnesses like HIV AIDS")

In [69]:
from llama_index.legacy.response.pprint_utils import pprint_response
pprint_response(response,show_source=True)

Final Response: According to Ayurveda, HIV/AIDS is a condition called
"Dhaatugata Jvara," which is characterized by a weakened immune
system. Ayurveda recommends a tissue-building approach (Santarpan`a
Chikitsaa) to restore the immune system and prevent opportunistic
infections. Herbs like Ashwagandha, Amla, and Guduchi are used in this
therapy. S`hodhana therapy (purification) is advised only for
asymptomatic patients and should be combined with Santarpan`a
Chikitsaa. The choice of herbs depends on the affected body system
(Srotas).
______________________________________________________________________
Source Node 1/3
Node ID: f0153eec-5bad-435c-be45-fcd89561dc29
Similarity: 0.7794220610267687
Text: Ayurvedic medicine, a traditional Indian system of medicine,
emphasizes the balance of three doshas: Vata, Pitta, and Kapha. These
doshas are believed to govern various functions in the body, and
imbalances can lead to disease. Ayurveda also places great importance
on diet and lifestyle, w

In [66]:
store.get_document('f0153eec-5bad-435c-be45-fcd89561dc29').get_content(MetadataMode.LLM)

'tag: p\nfile_path: /home/dai/rag/data/clean_html/Articles/Stree_Roga_Prasuti_Tantra_Baala_Roga/Somaroga_Nidaana.htm\nsummary: Reference: M Ni. Paris`hisht`a Nirukti: Sandhaaran`aat S`hareerasya Taa Aapah Soma Samdnyitaah | Tatah Somakshayaat Streen`aam Somaroga Iti Smritah || M Ni. Paris`hisht`a As body is sustained due to body fluids, it is called as Soma. This Soma in females gets depleted through the urinary tract; therefore this disease is called as Somaroga. Hetu: Sampraapti: Symptoms :\nis_summary_swapped: True\n\nAyurvedic medicine, a traditional Indian system of medicine, emphasizes the balance of three doshas: Vata, Pitta, and Kapha. These doshas are believed to govern various functions in the body, and imbalances can lead to disease. Ayurveda also places great importance on diet and lifestyle, with specific recommendations for different body types and conditions.'

----

In [ ]:
Hi, how can I help you?

how can i include some metadata to pass to the llm while using query engine from vectorstore

To include metadata while using the query engine from VectorStore, you can use the MetadataFilters and ExactMatchFilter classes from the llama_index.vector_stores.types module.

Here is an example of how you can do this:

from llama_index.vector_stores.types import ExactMatchFilter, MetadataFilters

query_engine = index.as_query_engine(
    similarity_top_k=3,
    vector_store_query_mode="default",
    filters=MetadataFilters(
        filters=[
            ExactMatchFilter(key="name", value="paul graham"),
        ]
    ),
    alpha=None,
    doc_ids=None,
)
response = query_engine.query("what did the author do growing up?")
In this example, the ExactMatchFilter is used to filter the documents based on the metadata key "name" and value "paul graham". The MetadataFilters class is used to pass these filters to the query engine.

You can customize the key and value in ExactMatchFilter to match the metadata you want to filter on.

Was this response helpful?

Yes
No
Verified Sources:

1. Vector Store Guide - Indexing
2. Tencent Vector DBIndex Demo - Vecto...
3. Chroma Auto Retriever - Vector Stor...
4. Metadata Extraction SEC - Metadata ...
so what i want to do is while querying i want to pass metadata['summary'] to llm for generation

To pass the summary metadata to the LLM for generation while querying, you can use the prompt_template field in the LLM predictor. The prompt_template field allows you to specify a template for the prompt that is sent to the LLM for generation. You can include the summary metadata in this template.

Here is an example of how you can do this:

from llama_index.llm_predictor.base import BaseLLMPredictor

class MyLLMPredictor(BaseLLMPredictor):
    prompt_template = 'Here is the summary of the section:\n{summary}\n\nGenerate a response based on this summary.'

    def generate_prompt(self, node, query):
        summary = node.metadata.get('summary', '')
        return self.prompt_template.format(summary=summary)
In this example, a new LLM predictor class MyLLMPredictor is created. The prompt_template field is set to a template that includes a placeholder for the summary metadata. The generate_prompt method is overridden to get the summary metadata from the node and format the prompt_template with this summary.

You can then use this LLM predictor when creating your query engine:

query_engine = index.as_query_engine(llm_predictor=MyLLMPredictor())
This will pass the summary metadata to the LLM for generation when querying.

Was this response helpful?